In [1]:
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git

d:\AIChallenge\data_sotuyen1\dataset_extraction\transnet\TransNetV2\inference


Cloning into 'TransNetV2'...
Updating files:  48% (15/31)
Updating files:  51% (16/31)
Updating files:  54% (17/31)
Updating files:  58% (18/31)
Updating files:  61% (19/31)
Updating files:  64% (20/31)
Updating files:  67% (21/31)
Updating files:  70% (22/31)
Updating files:  74% (23/31)
Updating files:  77% (24/31)
Updating files:  80% (25/31)
Updating files:  83% (26/31)
Updating files:  87% (27/31)
Updating files:  90% (28/31)
Updating files:  93% (29/31)
Updating files:  96% (30/31)
Updating files: 100% (31/31)
Updating files: 100% (31/31), done.
Filtering content:  66% (2/3)
Filtering content: 100% (3/3), 5.66 MiB | 1.44 MiB/s
Filtering content: 100% (3/3), 34.77 MiB | 3.15 MiB/s, done.


In [1]:
%cd TransNetV2/inference

d:\AIChallenge\data_sotuyen1\dataset_extraction\transnet\TransNetV2\inference


In [2]:
# Import module
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2

# Parse video info

In [4]:
videos_dir = 'D:/AIChallenge/data_sotuyen1/video'
all_video_paths = dict()
for part in sorted(os.listdir(videos_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_video_paths[data_part] =  dict()

for data_part in sorted(all_video_paths.keys()):
    data_part_path = f'{videos_dir}/Videos_{data_part}/video'
    video_paths = sorted(os.listdir(data_part_path))
    video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
    for video_id, video_path in zip(video_ids, video_paths):
        video_path_full = f'{data_part_path}/{video_path}'
        all_video_paths[data_part][video_id] = video_path_full

In [5]:
print( all_video_paths)

{'L18': {}, 'L19': {'V001': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V001.mp4', 'V003': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V003.mp4', 'V004': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V004.mp4', 'V005': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V005.mp4', 'V006': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V006.mp4', 'V007': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V007.mp4', 'V008': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V008.mp4', 'V009': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V009.mp4', 'V010': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V010.mp4', 'V011': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V011.mp4', 'V012': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V012.mp4', 'V013': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V013.mp4', 'V014': 'D:/AIChallenge/data_sotuyen1/video/Videos_L19/video/L19_V01

# Inference

In [6]:
import concurrent.futures

model = TransNetV2()  # Khởi tạo model
save_dir = 'D:/AIChallenge/data_sotuyen1/dataset_extraction/transnet/SceneJSON'
os.makedirs(save_dir, exist_ok=True)

def process_video(video_info):
    key, video_id, video_path = video_info
    # Dự đoán khung hình và các cảnh từ video
    _, single_frame_predictions, _ = model.predict_video(video_path)
    scenes = model.predictions_to_scenes(single_frame_predictions)
    
    # Tạo thư mục cho từng key
    key_dir = os.path.join(save_dir, key)
    os.makedirs(key_dir, exist_ok=True)
    
    # Lưu thông tin cảnh vào file JSON
    with open(f"{key_dir}/{video_id}.json", 'w') as f:
        json.dump(scenes.tolist(), f)

# Lọc các video trong thư mục L22 từ V019 trở đi
video_info_list = [
    (key, video_id, video_paths_dict[video_id])
    for key, video_paths_dict in all_video_paths.items()
    if key == 'L22'  # Chỉ xử lý thư mục L22
    for video_id in sorted(video_paths_dict.keys())
    if video_id >= 'V020'  # Chỉ xử lý các video từ V019 trở đi
]

# Sử dụng ThreadPoolExecutor để xử lý song song
with concurrent.futures.ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(process_video, video_info_list), total=len(video_info_list)))


[TransNetV2] Using weights from d:\AIChallenge\data_sotuyen1\dataset_extraction\transnet\TransNetV2\inference\transnetv2-weights/.
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V020.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V021.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V022.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V023.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V024.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V025.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V026.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuyen1/video/Videos_L22/video/L22_V027.mp4
[TransNetV2] Extracting frames from D:/AIChallenge/data_sotuy

  0%|          | 0/12 [00:00<?, ?it/s]

[TransNetV2] Processing video frames 27945/27945
[TransNetV2] Processing video frames 29754/29754
[TransNetV2] Processing video frames 31066/31066
[TransNetV2] Processing video frames 31679/31679
[TransNetV2] Processing video frames 31705/31705
[TransNetV2] Processing video frames 33445/33445
[TransNetV2] Processing video frames 33200/33200
[TransNetV2] Processing video frames 33875/33875
[TransNetV2] Processing video frames 34149/34149
[TransNetV2] Processing video frames 34064/34064
[TransNetV2] Processing video frames 38124/38124
[TransNetV2] Processing video frames 38981/38981


100%|██████████| 12/12 [1:54:01<00:00, 570.16s/it]   


In [1]:
!rm -r ./TransNetV2

'rm' is not recognized as an internal or external command,
operable program or batch file.
